# Extractive question answering

In [76]:
from transformers import AutoTokenizer
import torch
from transformers import AutoModelForQuestionAnswering

# model_ckpt = "albert-xxlarge-v2" 
# model_ckpt = "deepset/minilm-uncased-squad2"
model_ckpt = "gaussalgo/xlm-roberta-large_extractive-QA_en-cs"
# model_ckpt = "gaussalgo/mt5-large-priming-QA_en-cs"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModelForQuestionAnswering.from_pretrained(model_ckpt)

In [77]:
questions = ["Co se vám líbilo?", "Co se vám nelíbilo?", "Jaké byly postavy?", "Jaký byl děj?", "Jaké máte pocity z filmu?", "Jste spokojeni nebo zklamáni?", "Co si myslíte o Cameronovi?", "Co jste měli dnes k obědu?"]
context = """Kinoprojekce 3D. Moment překvapení již sice vyprchal, ale přesto jde stále o bohatou podívanou s propracovanou logistikou. Nejvíce mě nadchla ukázka fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk s podpisem skvělého režiséra mě opět vyvedl na 3 hodiny z tvrdé reality pozemšťana s výjimkou bojových a ziskuchtivých scén, jež jsou buhužel až příliš aktuální a velmi destruktivní pro lidskou populaci…😢😏😕🤪"""

In [78]:
print(tokenizer.decode(inputs["input_ids"][0]))

<s> Co jste měli dnes k obědu?</s></s> Kinoprojekce 3D. Moment překvapení již sice vyprchal, ale přesto jde stále o bohatou podívanou s propracovanou logistikou. Nejvíce mě nadchla ukázka fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk s podpisem skvělého režiséra mě opět vyvedl na 3 hodiny z tvrdé reality pozemšťana s výjimkou bojových a ziskuchtivých scén, jež jsou buhužel až příliš aktuální a velmi destruktivní pro lidskou populaci...😢😏😕🤪</s>


In [79]:
print(f"Context: {context}")

for question in questions:
    inputs = tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**inputs)
    start_logits = outputs.start_logits
    end_logits = outputs.end_logits
    start_idx = torch.argmax(start_logits)  
    end_idx = torch.argmax(end_logits) + 1  
    answer_span = inputs["input_ids"][0][start_idx:end_idx]
    answer = tokenizer.decode(answer_span)
    pipe = pipeline("question-answering", model=model, tokenizer=tokenizer)
    answers = pipe(question=question, context=context, topk=3, handle_impossible_answer=True)
    
    print(f"Question: {question}")
    for idx, answer in enumerate(answers):
        print(f"\tTop {idx+1} answer: {answer['answer']}")

Context: Kinoprojekce 3D. Moment překvapení již sice vyprchal, ale přesto jde stále o bohatou podívanou s propracovanou logistikou. Nejvíce mě nadchla ukázka fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk s podpisem skvělého režiséra mě opět vyvedl na 3 hodiny z tvrdé reality pozemšťana s výjimkou bojových a ziskuchtivých scén, jež jsou buhužel až příliš aktuální a velmi destruktivní pro lidskou populaci…😢😏😕🤪
Question: Co se vám líbilo?
	Top 1 answer: fantazie mořského světa s propojením duše a přírody.😍👏Majstrštyk
	Top 2 answer: fantazie mořského světa s
	Top 3 answer: Kinoprojekce 3D.
Question: Co se vám nelíbilo?
	Top 1 answer: Kinoprojekce 3D.
	Top 2 answer: 
	Top 3 answer: bojových a ziskuchtivých scén,
Question: Jaké byly postavy?
	Top 1 answer: 
	Top 2 answer: buhužel
	Top 3 answer: buhužel až
Question: Jaký byl děj?
	Top 1 answer: 
	Top 2 answer: fantazie
	Top 3 answer: fantazie mořského
Question: Jaké máte pocity z filmu?
	Top 1 answer: překvapení již
	Top 2 

# Generative question answering

In [87]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("gaussalgo/mt5-large-priming-QA_en-cs")
model = AutoModelForSeq2SeqLM.from_pretrained("gaussalgo/mt5-large-priming-QA_en-cs")


input_text = """
    Question: Jak se jmenuje prezident? 
    Context: Včera jsem měl rozhovor s nově zvoleným prezidentem, Petrem Pavlem.
    Answer: Petr Pavel 
    Question: Při studiu jsem absolvoval mnoho nudných předkášek o dějinách ekonomického myšlení.
    Context: Jaké byly přednášky z dějin ekonomického myšlení?
    Answer: nudné
    Question: Dnes jsem měl k obědu kuřecí kung pao.
    Context: Co jsem měl dnes k obědu?
    Answer: kuřecí kung pao
    Question: Moje oblíbená sci-fi kniha je Enderova hra.
    Context: Jaká je moje oblíbená sci-fi kniha?
    Answer: Enderova hra.
    Question: Nejraději ve volném čase čtu.
    Context: Co dělám nejraději ve volném čase?
    Answer:"""

# For the expected format of input_text, see Intended use above
inputs = tokenizer(input_text, return_tensors="pt")

outputs = model.generate(**inputs)

print(f'Input text: {input_text}')
print("Answer:")
print(tokenizer.decode(outputs[0]))

Input text: 
    Question: Jak se jmenuje prezident? 
    Context: Včera jsem měl rozhovor s nově zvoleným prezidentem, Petrem Pavlem.
    Answer: Petr Pavel 
    Question: Při studiu jsem absolvoval mnoho nudných předkášek o dějinách ekonomického myšlení.
    Context: Jaké byly přednášky z dějin ekonomického myšlení?
    Answer: nudné
    Question: Dnes jsem měl k obědu kuřecí kung pao.
    Context: Co jsem měl dnes k obědu?
    Answer: kuření kung pao
    Question: Moje oblíbená sci-fi kniha je Enderova hra.
    Context: Jaká je moje oblíbená sci-fi kniha?
    Answer: Enderova hra.
    Question: Nejraději ve volném čase čtu.
    Context: Co dělám nejraději ve volném čase?
    Answer:
Answer:
<pad> čtu</s>


In [85]:
print(tokenizer.decode(outputs[0]))

<pad> čtu</s>
